# Example: Parallel Network using LFPy

This example notebook submits a job to a chosen compute cluster (**JUSUF**), loads a **Singularity** container with a working **LFPy** installation and executes a version of the LFPy example file ``example_network.py`` (https://github.com/LFPy/LFPy/blob/master/examples/example_network/example_network.py) in parallel on one HPC backend of EBRAINS/HBP.

The example demonstrates usage of ``LFPy.Network`` with network of ball-and-stick type
morphologies with active HH channels inserted in the somas and passive-leak
channels distributed throughout the apical dendrite. The corresponding
morphology and template specifications are in the files ``BallAndStick.hoc`` and
``BallAndStickTemplate.hoc``.


For more indepth info on using HPC backends via EBRAINS see e.g., https://wiki.ebrains.eu/bin/view/Collabs/using-supercomputers-from-the-collab/

## Prepare job

In [1]:
# use the pyunicore library
!pip install pyunicore==0.9.12 --upgrade

Requirement already up-to-date: pyunicore==0.9.12 in /opt/app-root/lib/python3.6/site-packages
Requirement already up-to-date: PyJWT>=1.7 in /opt/app-root/lib/python3.6/site-packages (from pyunicore==0.9.12)
Requirement already up-to-date: requests>=2.5 in /opt/app-root/lib/python3.6/site-packages (from pyunicore==0.9.12)
Requirement already up-to-date: charset-normalizer~=2.0.0; python_version >= "3" in /opt/app-root/lib/python3.6/site-packages (from requests>=2.5->pyunicore==0.9.12)
Requirement already up-to-date: idna<4,>=2.5; python_version >= "3" in /opt/app-root/lib/python3.6/site-packages (from requests>=2.5->pyunicore==0.9.12)
Requirement already up-to-date: urllib3<1.27,>=1.21.1 in /opt/app-root/lib/python3.6/site-packages (from requests>=2.5->pyunicore==0.9.12)
Requirement already up-to-date: certifi>=2017.4.17 in /opt/app-root/lib/python3.6/site-packages (from requests>=2.5->pyunicore==0.9.12)
You are using pip version 9.0.1, however version 21.3.1 is available.
You should c

In [2]:
# import modules
import os
import pyunicore.client as unicore_client
import requests
import json
from time import sleep, time
from pprint import pprint
from IPython.display import IFrame, clear_output

In [3]:
# Create connection to supercomputer (i.e., JUSUF)
tr = unicore_client.Transport(clb_oauth.get_token())
r = unicore_client.Registry(tr, unicore_client._HBP_REGISTRY_URL)

In [4]:
# Valid choices for supercomputers are one of the keys in:
r.site_urls

{'JUWELS': 'https://zam2125.zam.kfa-juelich.de:9112/JUWELS/rest/core',
 'JUSUF': 'https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core',
 'UNICORE-TEST': 'https://catto.cscs.ch:8080/UNICORE-TEST/rest/core',
 'BSC-MareNostrum': 'https://unicore-hbp.bsc.es:8080/BSC-MareNostrum/rest/core',
 'JURECA': 'https://zam2125.zam.kfa-juelich.de:9112/JURECA/rest/core',
 'DAINT-CSCS': 'https://brissago.cscs.ch:8080/DAINT-CSCS/rest/core',
 'CINECA-MARCONI': 'https://grid.hpc.cineca.it:9111/CINECA-MARCONI/rest/core',
 'CINECA-G100': 'https://grid.hpc.cineca.it:9111/CINECA-G100/rest/core',
 'CINECA-M100': 'https://grid.hpc.cineca.it:9111/CINECA-M100/rest/core',
 'JUDAC': 'https://zam2125.zam.kfa-juelich.de:9112/JUDAC/rest/core'}

In [5]:
# connect to HPC resouce - change subpercomputer name to whatever you have access to:
supercomputer = 'JUSUF'
try:
    site_client = r.site(supercomputer)
except KeyError:
    # cluster may be dropped from Registry.site_urls for whatever reason
    site_client = unicore_client.Client(r.transport, 'https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core')

In [6]:
site_client.access_info()

{'role': {'selected': 'user', 'availableRoles': ['user']},
 'queues': {'availableQueues': ['batch', 'develgpus'], 'selected': 'batch'},
 'dn': 'UID=espen.hagen@nmbu.no',
 'xlogin': {'UID': 'hagen2',
  'availableGroups': ['icei-hbp-2020-0004'],
  'availableUIDs': ['hagen2'],
  'group': 'icei-hbp-2020-0004'}}

In [7]:
# check connection to supercomputer
headers = {}
headers["Authorization"] = "Bearer " + clb_oauth.get_token()
headers['Accept'] = "application/json"
rs = requests.get(url=site_client.site_url, headers = headers, verify = False)
print("Status code %s " % rs.status_code)
print("Content-type %s " % rs.headers['Content-Type'])
reply = rs.json()
# print(json.dumps(reply, indent = 1))

Status code 200 
Content-type application/json 


## Download LFPy example network files
Prepare simulation files using example files from the main LFPy repository (https://github.com/LFPy/LFPy.git)

In [11]:
!wget -O example_network.py https://raw.githubusercontent.com/LFPy/LFPy/master/examples/example_network/example_network.py
!wget -O BallAndStick.hoc https://raw.githubusercontent.com/LFPy/LFPy/master/examples/example_network/BallAndStick.hoc
!wget -O BallAndStickTemplate.hoc https://raw.githubusercontent.com/LFPy/LFPy/master/examples/example_network/BallAndStickTemplate.hoc

--2021-11-22 12:13:55--  https://raw.githubusercontent.com/LFPy/LFPy/master/examples/example_network/example_network.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20130 (20K) [text/plain]
Saving to: ‘example_network.py’

100%[======================================>] 20,130      --.-K/s   in 0.002s  

2021-11-22 12:13:56 (8.64 MB/s) - ‘example_network.py’ saved [20130/20130]

--2021-11-22 12:13:56--  https://raw.githubusercontent.com/LFPy/LFPy/master/examples/example_network/BallAndStick.hoc
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 20

In [12]:
## to make things more interesting with HPCs, let's increase the network size and adjust synaptic weights by applying a patch:
diff = '''190c190
< population_sizes = [256, 64]
---
> population_sizes = [2048, 512]
207,210c207,210
< weightArguments = [[dict(loc=0.001, scale=0.0001),
<                     dict(loc=0.001, scale=0.0001)],
<                    [dict(loc=0.01, scale=0.001),
<                     dict(loc=0.01, scale=0.001)]]
---
> weightArguments = [[dict(loc=0.0005, scale=0.0001),
>                     dict(loc=0.0005, scale=0.0001)],
>                    [dict(loc=0.005, scale=0.001),
>                     dict(loc=0.005, scale=0.001)]]
'''

with open('patch.diff', 'w') as f:
    f.writelines(diff)

In [13]:
# apply patch
!patch example_network.py patch.diff

patching file example_network.py


## Prepare singularity container
See https://gitlab.version.fz-juelich.de/bvonstvieth_publications/container_userdoc_tmp for details. 

This step builds the singularity container. It is optional if the recipe has already been uploaded and built on the system. 

The procedure may be different on different HPC backends. 
The container can either way be built from the same recipe: https://raw.githubusercontent.com/LFPy/LFPy/2.2.dev0/Dockerfile

Turn the below Raw block into Code in order to run:

## Prepare main simulation job
This step combines in a single session the following:

- download LFPy container
- upload simulation files from the collab
- ask for resources (# nodes, # MPI processes, # seconds runtime)
- execute simulation
- zip simulation output

In [43]:
# create dictionary with job specification and define list of input files from this Collab
# simulation_job = {"Job type": "interactive"}
simulation_job = {}
simulation_inputs = ["example_network.py", "BallAndStick.hoc", "BallAndStickTemplate.hoc"] 

In [44]:
# Resources
simulation_job['Resources'] = {
    "Queue": "batch",
    #"CPUs": "256",
    "Nodes": "2",
    "CPUsPerNode": "128",
    "Runtime": "600",
}

In [45]:
# commands run on login node before job execution
simulation_job["User precommand"] = """module use $OTHERSTAGES
module --force purge
module load Stages/2020
module load GCC Singularity-Tools
sib download --recipe-name lfpymaster
"""
simulation_job["RunUserPrecommandOnLoginNode"] = "true"

In [46]:
# - set some environment variables
# - run the python code using interpreter embedded in container
simulation_job["Executable"] = """module use $OTHERSTAGES
module --force purge
module load Stages/2020
module load GCC Singularity-Tools
unset DISPLAY  # matplotlib may look for a nonexistant display on compute node 
srun --mpi=pmi2 singularity exec lfpymaster.sif python3 -u example_network.py
"""

In [47]:
# commands run after job is done
simulation_job["User postcommand"] = "tar -cvf example_network_output.tar example_network_output" 
simulation_job["RunUserPostcommandOnLoginNode"] = "true"

In [48]:
simulation_job

{'Resources': {'Queue': 'batch',
  'Nodes': '2',
  'CPUsPerNode': '128',
  'Runtime': '600'},
 'User precommand': 'module use $OTHERSTAGES\nmodule --force purge\nmodule load Stages/2020\nmodule load GCC Singularity-Tools\nsib download --recipe-name lfpymaster\n',
 'RunUserPrecommandOnLoginNode': 'true',
 'Executable': 'module use $OTHERSTAGES\nmodule --force purge\nmodule load Stages/2020\nmodule load GCC Singularity-Tools\nunset DISPLAY  # matplotlib may look for a nonexistant display on compute node \nsrun --mpi=pmi2 singularity exec lfpymaster.sif python3 -u example_network.py\n',
 'User postcommand': 'tar -cvf example_network_output.tar example_network_output',
 'RunUserPostcommandOnLoginNode': 'true'}

In [49]:
# create job
job = site_client.new_job(job_description=simulation_job, inputs=simulation_inputs)

In [50]:
job.properties

{'owner': 'UID=espen.hagen@nmbu.no',
 'submissionPreferences': {'UC_OAUTH_BEARER_TOKEN': ['eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJfNkZVSHFaSDNIRmVhS0pEZDhXcUx6LWFlZ3kzYXFodVNJZ1RXaTA1U2k0In0.eyJleHAiOjE2MzgxNzczMzMsImlhdCI6MTYzNzU3MjUzMywiYXV0aF90aW1lIjoxNjM3NTY3NTAyLCJqdGkiOiI3MWRkNTNlZS00MDAxLTQ5NzMtODMzYy1kYmFiMmQyODM1NTYiLCJpc3MiOiJodHRwczovL2lhbS5lYnJhaW5zLmV1L2F1dGgvcmVhbG1zL2hicCIsImF1ZCI6WyJpbWdfc3ZjIiwib3BlbnNoaWZ0IiwiaGFyYm9yIiwidGVhbSIsInBsdXMiLCJqdXB5dGVyaHViLW9wZW5zaGlmdC1wcmV2aWV3IiwianVweXRlcmh1YiIsInh3aWtpIiwib3BlbnNoaWZ0LWpzYyIsImFjY291bnQiLCJvcGVuc2hpZnQtZGV2IiwiZ3JvdXAiXSwic3ViIjoiMDVmOGVjYzMtNDkzYi00OTdlLTkyMDktNDYxMTdlNGYyNWVmIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoianVweXRlcmh1Yi1qc2MiLCJzZXNzaW9uX3N0YXRlIjoiYTk3N2NiOGUtOTczYy00Yzc2LWE1N2ItMTFjMDlkZWE0MGFiIiwiYWNyIjoiMCIsImFsbG93ZWQtb3JpZ2lucyI6WyJodHRwczovL2xhYi5qc2MuZWJyYWlucy5ldSJdLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiY29sbGFib3JhdG9yeV9tZW1iZXIiLCJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIl19LCJzY29wZSI6I

In [51]:
# wait while job is running and track progress
while job.is_running():
    clear_output()
    # STDERR output (if any)
    try:
        stderr = job.working_dir.stat('stderr')
        #pprint(stderr.raw().readlines()[:5])
        pprint(stderr.raw().readlines()[-5:])
    except:
        print('no stderr file to read yet')
    # STDOUT output
    try:
        stdout = job.working_dir.stat('stdout')
        #pprint(stdout.raw().readlines()[:10])
        pprint(stdout.raw().readlines()[-10:])
    except:
        print('no stdout file to read yet')
    sleep(10)

[b'  warn(mssg)\n',
 b'/usr/local/lib/python3.9/dist-packages/LFPy/cell.py:182: UserWarning: 18 sec'
 b"tions detected! Consider setting 'delete_sections=True'\n",
 b'  warn(mssg)\n',
 b'/usr/local/lib/python3.9/dist-packages/LFPy/cell.py:182: UserWarning: 18 sec'
 b"tions detected! Consider setting 'delete_sections=True'\n",
 b'  warn(mssg)\n']
[b't = 300.0 ms\n',
 b't = 400.0 ms\n',
 b't = 500.0 ms\n',
 b't = 600.0 ms\n',
 b't = 700.0 ms\n',
 b't = 800.0 ms\n',
 b't = 900.0 ms\n',
 b't = 1000.0 ms\n',
 b't = 1100.0 ms\n',
 b't = 1200.0 ms\n']


In [56]:
job.working_dir.listdir().keys()

dict_keys(['example_network_output/', 'stderr', 'BallAndStickTemplate.hoc', 'example_network.py', 'bss_submit_1637580709170', 'example_network_output.tar', 'stdout', '.UNICORE_POST_0/', '.UNICORE_PRE_0/', 'UNICORE_Job_1637580709170', 'UNICORE_SCRIPT_EXIT_CODE', 'BallAndStick.hoc', 'lfpymaster.sif'])

In [57]:
# STDERR output (if any)
stderr = job.working_dir.stat('stderr')
pprint(stderr.raw().readlines()[:5])
pprint(stderr.raw().readlines()[-5:])

[b'\n',
 b'  Preparing the environment for use of 2020 stage.\n',
 b'\n',
 b'\n',
 b'  Preparing the environment for use of requested stage ( 2020 ).\n']
[b'  warn(mssg)\n',
 b'/usr/local/lib/python3.9/dist-packages/LFPy/cell.py:182: UserWarning: 18 sec'
 b"tions detected! Consider setting 'delete_sections=True'\n",
 b'  warn(mssg)\n',
 b'/usr/local/lib/python3.9/dist-packages/LFPy/cell.py:182: UserWarning: 18 sec'
 b"tions detected! Consider setting 'delete_sections=True'\n",
 b'  warn(mssg)\n']


In [58]:
# STDOUT output
stdout = job.working_dir.stat('stdout')
pprint(stdout.raw().readlines()[:10])
pprint(stdout.raw().readlines()[-10:])

[b'numprocs=256\n',
 b'Connected population E to E by 419479 connections and 838923 synapses\n',
 b'Connected population E to I by 104392 connections and 208730 synapses\n',
 b'Connected population I to E by 105115 connections and 525650 synapses\n',
 b'Connected population I to I by 26100 connections and 130749 synapses\n',
 b't = 100.0 ms\n',
 b't = 200.0 ms\n',
 b't = 300.0 ms\n',
 b't = 400.0 ms\n',
 b't = 500.0 ms\n']
[b't = 300.0 ms\n',
 b't = 400.0 ms\n',
 b't = 500.0 ms\n',
 b't = 600.0 ms\n',
 b't = 700.0 ms\n',
 b't = 800.0 ms\n',
 b't = 900.0 ms\n',
 b't = 1000.0 ms\n',
 b't = 1100.0 ms\n',
 b't = 1200.0 ms\n']


In [59]:
# download simulation output
job.working_dir.stat('example_network_output.tar').download('example_network_output.tar')

In [60]:
# kill job, clean up files on the remote
job.delete()

## Investigate simulation output

In [61]:
# untar output to folder example_network_output
!tar -xf example_network_output.tar

In [62]:
# cell morphologies on one particular MPI process
IFrame("./example_network_output/population_RANK_0.pdf", width=800, height=600)

In [63]:
# spike raster plot
IFrame("./example_network_output/spike_raster.pdf", width=800, height=600)

In [64]:
# Extracellular potentials of each population and in total
IFrame("./example_network_output/extracellular_potential.pdf", width=800, height=600)

In [65]:
# Current dipole moment
IFrame("./example_network_output/current_dipole_moment.pdf", width=800, height=600)